In [1]:
from dotenv import load_dotenv 

load_dotenv() 

True

In [ ]:
import chromadb

chroma_client = chromadb.PersistentClient(path="chromadb")

In [ ]:
import os
from supabase import create_client, Client

supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")
supabase_client = create_client(supabase_url, supabase_key)

## Recommend Article

In [ ]:
# Get user preference from database

user_info = supabase_client.table("buddy").select("preferred_topic").eq('buddy_id', 1).execute()
preferred_topics = user_info.model_dump()['data'][0]['preferred_topic']

In [ ]:
preferred_topics

['Finance', 'Economy', 'Environment']

In [ ]:
# Vector Database

article_collection = chroma_client.get_collection('article')

In [ ]:
# Construct Query Sentence

query_sentences = "articles about " + ", ".join(preferred_topics)
query_sentences

'articles about Finance, Economy, Environment'

In [ ]:
result = article_collection.query(
            query_texts=query_sentences,
            n_results=10,
        )

result

## Recommend Buddy

In [ ]:
# Vector Database

buddy_collection = chroma_client.get_collection('buddy')

In [ ]:
# Get User Description

user = buddy_collection.get("1")
user_description = user['documents'][0]
user_description

In [ ]:
def create_buddy_sentence(buddy):
    return f"A buddy from {buddy['role']}, with interest in {buddy['professional_interest']} " \
           f"likes {', '.join(eval(buddy['hobby']))}. proficient in { ', '.join(eval(buddy['language']))} " \
           f"with a goal in {buddy['buddy_program_goal']}"

In [ ]:
# Get the buddy with the most similar description

result = buddy_collection.query(
        query_texts=user_description, # Use the user_description as the query text
        n_results=1,
        where={
            "$and" : [
                {"years_of_experience" : {"$gte" : user['metadatas'][0]['years_of_experience'] - 2}},      
                {"years_of_experience" : {"$lte" : user['metadatas'][0]['years_of_experience'] + 2}},

                {"interaction_frequency" : user['metadatas'][0]['interaction_frequency']},

                {"meeting_preference" :  user['metadatas'][0]['meeting_preference']},

                {"email" :  {"$ne" : user['metadatas'][0]['email']}},
            ]
        }
    )

result

## Recommend Mentor

In [ ]:
# Vector Database

mentor_collection = chroma_client.get_collection('mentor')

In [ ]:
def create_mentor_sentence(mentor):
    return f"A mentor from {mentor['department']} department, working as a {mentor['position']} " \
           f"specializing in {mentor['specialization']}. {mentor['short_desc']}"

In [ ]:
# Will be obtained from user survey

mentor_request = "I want a mentor that works in the Finance department and has experience in Financial Modelling"

In [ ]:
result = mentor_collection.query(
            query_texts=mentor_request,
            n_results=1,
        )

result